# Step 4: Feature Engineering
Classic Machine Learning models rely on _[Features](https://en.wikipedia.org/wiki/Feature_(machine_learning))_. 
A _feature_ is an independent and measurable input to our model.
_[Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)_ is the process of extracting this information from raw data based on domain knowledge.

## Environment Setup
You know what to do....

In [ ]:
!pip install -U -r requirements.txt

In [ ]:
import os 
import trino 
import pandas 
from helper import get_sql

In [ ]:
TRINO_HOSTNAME = os.environ.get('TRINO_HOSTNAME')
TRINO_USERNAME = os.environ.get('TRINO_USERNAME')
TRINO_PORT = os.environ.get('TRINO_PORT')

In [ ]:
conn = trino.dbapi.connect(
    host=TRINO_HOSTNAME,
    port=TRINO_PORT,
    user=TRINO_USERNAME,
)

In [ ]:
sql = 'SHOW CATALOGS'
df = get_sql(sql, conn)
df.head()

## Join Siloed Data
Let's start by recreating the our DataFrame from the last notebook.
We used `pandas` to join the `customer` and `finance` DataFrames.
Let's use **Starburst** to create the same dataset.

In [ ]:
# basic join between customer-domain and finance-domain
# similar to what we did in the previous notebook
sql = '''

SELECT 
    c.id, 
    c.customername, 
    c.customeraddr, 
    c.mktsegment, 
    c.status, 
    SUM(o.amount) as sum_purchased
FROM "customer-domain".public.customer c 
JOIN "finance-domain".public.transactions o ON c.id = o.customerid
GROUP BY c.id, c.customername, c.customeraddr, c.mktsegment, c.status
ORDER BY sum_purchased DESC

'''

df = get_sql(sql, conn)
df.head()

Great.
Now we can layer in basic stats from our chatbot.
Let's take the `count()` of messages by customer and join to the dataset we just built.
We do this using `WITH`, a SQL sub-query that creates temporary tables to facillitate calculations.

**Please note:** it's easy to introduce performance issues when using `WITH`.
Please read up on it before incorporating into your workflow!

In [ ]:
# clean and extract JSON message on kafka queue first 
# then LEFT JOIN to customer and finance domains
sql = '''

WITH 
    KAFKA AS (
        SELECT 
            id, 
            COUNT(*) AS message_count
        FROM (
            SELECT 
                JSON_EXTRACT(_message, '$.customer_number') as id, 
                JSON_EXTRACT(_message, '$.txt') as txt 
            FROM "messages"
        )
        GROUP BY id 
    ) 

SELECT DISTINCT
    c.id, 
    c.customername, 
    c.mktsegment, 
    SUM(o.amount) AS sum_purchased, 
    k.message_count
FROM "customer-domain".public.customer c 
JOIN "finance-domain".public.transactions o ON c.id = o.customerid
LEFT JOIN KAFKA k on c.id = CAST(k.id AS INTEGER)
GROUP BY c.id, c.customername, c.mktsegment, k.message_count
ORDER BY sum_purchased DESC

'''
df = get_sql(sql, conn)
df.head()

Great!
We now have a customer table with basic information about the customer, the total amount spent, and a count of messages sent to customer service via the chatbot.

## Build Features
It's a good start, but there are some quick, obvious features we'd like to look at.
Let's grab some descriptive statistics related to customer spend.

In [ ]:
# use SQL to build features accross domains without making extracts or copies
# limit to active customers for now 
sql = '''

WITH 
    KAFKA AS (
        SELECT 
            id, 
            COUNT(*) AS message_count
        FROM (
            SELECT 
                JSON_EXTRACT(_message, '$.customer_number') as id, 
                JSON_EXTRACT(_message, '$.txt') as txt 
            FROM "messages"
        )
        GROUP BY id 
    )  

SELECT DISTINCT 
    c.id, 
    c.customername, 
    c.mktsegment, 
    day(current_date - c.effectivedate) AS tot_days_active,
    MAX(o.amount) AS tot_max_prch,
    MIN(o.amount) AS tot_min_prch,
    AVG(o.amount) AS tot_mean_prch,
    COUNT(o.amount) AS tot_count_prch,
    SUM(o.amount) AS tot_sum_prch, 
    k.message_count AS tot_message_count
FROM "customer-domain".public.customer c 
JOIN "finance-domain".public.transactions o ON c.id = o.customerid
LEFT JOIN KAFKA k on o.customerid = CAST(k.id AS INTEGER)
WHERE c.status > 0
GROUP BY c.id, c.customername, c.mktsegment, k.message_count, day(current_date - c.effectivedate)
ORDER BY tot_sum_prch DESC


'''
df = get_sql(sql, conn)
df.head()

Great!
We now have a customer table with minimum, maximum, average, and total spend over each customer's history with our company.
We also included a count of transactions.

Unfortunately, looking at a customer's total history with a company doesn't give us direct insight into their _current_ behavior.
Let's trim that window to three years (still a long time, I know, but it's a start). 

We create new temporary tables and then join them together below.

Can you think of a better way to do this? 

In [ ]:
# use SQL to build features accross domains without making extracts or copies
# limit to active customers for now 
sql = '''

WITH 
    KAFKA AS (
        SELECT 
            id, 
            COUNT(*) AS message_count
        FROM (
            SELECT 
                JSON_EXTRACT(_message, '$.customer_number') as id, 
                JSON_EXTRACT(_message, '$.txt') as txt 
            FROM "messages"
        )
        GROUP BY id 
    ),
    customer AS (
        SELECT 
            c.id, 
            c.customername, 
            c.effectivedate, 
            c.mktsegment, 
            c.status, 
            o.transdate, 
            o.amount 
        FROM "customer-domain".public.customer c 
        JOIN "finance-domain".public.transactions o ON c.id = o.customerid
    ),
    total AS (
        SELECT DISTINCT 
            c.id, 
            c.customername, 
            c.mktsegment, 
            day(current_date - c.effectivedate) AS tot_days_active,
            MAX(c.amount) AS tot_max_prch,
            MIN(c.amount) AS tot_min_prch,
            AVG(c.amount) AS tot_mean_prch,
            COUNT(c.amount) AS tot_count_prch,
            SUM(c.amount) AS tot_sum_prch, 
            k.message_count AS tot_message_count
        FROM customer c
        LEFT JOIN KAFKA k on c.id = CAST(k.id AS INTEGER)
        WHERE c.status > 0
        GROUP BY c.id, c.customername, c.mktsegment, k.message_Count, day(current_date - c.effectivedate)
        ORDER BY tot_sum_prch DESC
    ),
    three AS ( 
        SELECT DISTINCT 
            c.id, 
            MAX(c.amount) AS three_max_prch,
            MIN(c.amount) AS three_min_prch,
            AVG(c.amount) AS three_mean_prch,
            COUNT(c.amount) AS three_count_prch,
            SUM(c.amount) as three_sum_prch
        FROM customer c
        WHERE c.status > 0 AND c.transdate > date '2018-01-01'
        GROUP BY c.id, c.customername, c.mktsegment, day(current_date - c.effectivedate)
        ORDER BY three_sum_prch DESC
    )
    
SELECT 
    t.*, 
    ttt.three_max_prch,
    ttt.three_min_prch,
    ttt.three_count_prch,
    ttt.three_sum_prch
FROM total t 
JOIN three ttt ON t.id = ttt.id
ORDER BY tot_sum_prch DESC 

'''
df = get_sql(sql, conn)
df.head()

We now have a fairly wide dataset we can use to test some hypotheses!

## Materializing Views
We can materialize this dataset to ensure we use the same data from now on. 

Materializing views have a number of benefits, but I'll only highlight a few here:
- new data will automatically be included
- other Data Scientists will have access to the same Features
- and most importantly, we **know** how these features were calculated

We won't need to recreate these features as we scale our Data Science team, and given the nature of these calculations, we'll likely reuse such straight-forward features in future models.

Let's create our views.  

In [ ]:
#
# we are using the HIVE connector to create views 
#

sql = '''
CREATE VIEW datalake.default.messages_agg AS
SELECT 
    id, 
    COUNT(*) AS message_count    
FROM (
    SELECT 
        JSON_EXTRACT(_message, '$.customer_number') as id, 
        JSON_EXTRACT(_message, '$.txt') as txt 
    FROM kafka.default."messages"
    )
GROUP BY id 
ORDER BY message_count DESC

'''
get_sql(sql, conn)

In [ ]:
sql = '''
CREATE VIEW datalake.default.customer AS 
SELECT 
    c.id, 
    c.customername, 
    c.effectivedate, 
    c.mktsegment, 
    c.status, 
    o.transdate, 
    o.amount 
FROM "customer-domain".public.customer c 
JOIN "finance-domain".public.transactions o ON c.id = o.customerid

'''
get_sql(sql, conn)

In [ ]:

sql = '''
CREATE VIEW datalake.default.customer_lifetime_features AS 
SELECT DISTINCT 
    c.id, 
    c.customername, 
    c.mktsegment, 
    day(current_date - c.effectivedate) AS tot_days_active,
    MAX(c.amount) AS tot_max_prch,
    MIN(c.amount) AS tot_min_prch,
    AVG(c.amount) AS tot_mean_prch,
    COUNT(c.amount) AS tot_count_prch,
    SUM(c.amount) AS tot_sum_prch, 
    k.message_count AS tot_message_count
FROM datalake.default.customer c
LEFT JOIN datalake.default.messages_agg k on c.id = CAST(k.id AS INTEGER)
WHERE c.status > 0
GROUP BY c.id, c.customername, c.mktsegment, k.message_Count, day(current_date - c.effectivedate)
ORDER BY tot_sum_prch DESC

'''
get_sql(sql, conn)

In [ ]:
sql = '''
CREATE VIEW datalake.default.customer_3y_features AS 
SELECT DISTINCT 
    c.id, 
    MAX(c.amount) AS three_max_prch,
    MIN(c.amount) AS three_min_prch,
    AVG(c.amount) AS three_mean_prch,
    COUNT(c.amount) AS three_count_prch,
    SUM(c.amount) as three_sum_prch
FROM datalake.default.customer c
WHERE c.status > 0 AND c.transdate > date '2018-01-01'
GROUP BY c.id, c.customername, c.mktsegment, day(current_date - c.effectivedate)
ORDER BY three_sum_prch DESC

'''
get_sql(sql, conn)

In [ ]:
sql = '''

CREATE VIEW datalake.default.basic_features AS
SELECT
    t.*,
    ttt.three_max_prch,
    ttt.three_min_prch,
    ttt.three_count_prch,
    ttt.three_sum_prch
FROM datalake.default.customer_lifetime_features t
JOIN datalake.default.customer_3y_features ttt ON t.id = ttt.id
ORDER BY tot_sum_prch DESC

'''
get_sql(sql, conn)

Let's take a quick look at the tables and data stored in our Data Lake.

In [ ]:
sql = '''        

SHOW TABLES FROM datalake.default

'''
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = '''

DESCRIBE datalake.default.basic_features

'''

df = get_sql(sql, conn)
df


In [ ]:
sql = '''

SELECT * FROM datalake.default.basic_features

'''

df = get_sql(sql, conn)
df.head()

## Next 
Now that we've had a chance to explore our data and materialize the features we believe will be useful in the future, let's get to modeling.

[5_nltk.ipynb](5_nltk.ipynb) gives you a chance to show off your Data Science expertise!